참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 06.008

모멘트 면적법<br>Moment area method

p. 270

### 문제에서 주어진 변수<br>Given Parameters

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB_m, L_BC_m = sy.symbols('L_{AB}[m], L_{BC}[m]', real=True, nonnegative=True)

s_d = {
    L_AB_m: 4,
    L_BC_m: 4,
}

L_AC_m = L_AB_m + L_BC_m

#### 하중<br>Load

In [ ]:
P_A_N, w_AB_N_m = sy.symbols('P_{A}[N], w_{AB}[N/m]', real=True)

s_d.update(
    {
        w_AB_N_m: -600,
    }
)


보를 그려주는 모듈을 `import` 할 수 있도록 준비.

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_AB_m.subs(s_d), 'text':'B'},
    {'x_m': L_AC_m.subs(s_d), 'text':'C'},
]

reaction_list = [
    {'x_m': 0},
]

dist_load_list = [
    {'x_begin_m': 0, 'x_end_m': L_AB_m.subs(s_d), 'text': 'w0'}
]


draw_diagrams.draw_beam(L_AC_m.subs(s_d), points_list, reaction_list, dist_load_list=dist_load_list)

In [ ]:
x_m = sy.symbols('x[m]', nonnegative=True)
x_A_m = 0
x_B_m = L_AB_m
x_C_m = L_AC_m

### 별해<br>Alternative

p. 264 eq. 6.9

$$
t_{A/C} = \left. \frac{M}{EI}Area \right\rvert _{C}^{A}\overline{x}_{/A}=0
$$

#### 집중하중 $P$에 의한 굽힘모멘트 면적<br>Bending moment area by concentrated load $P$

$$
BMD_P(x)=Px
$$

In [ ]:
BMD_P = P_A_N * x_m

$$BMDarea_P = \frac{1}{2}PL^2$$

In [ ]:
BMD_area_P = sy.integrate(BMD_P, (x_m, x_A_m, x_C_m))

In [ ]:
BMD_area_P

도심 거리<br>Centroid distance

In [ ]:
centroid_dist_P =  sy.integrate(BMD_P * x_m, (x_m, x_A_m, x_C_m))/ sy.integrate(BMD_P, (x_m, x_A_m, x_C_m))

In [ ]:
centroid_dist_P

선도 면적 $\times$ 도심 거리<br>Diagram area $\times$ centroid distance

$$t_{A/C_p} = \frac{1}{2}PL^2 \frac{2}{3}L$$

In [ ]:
t_A_C_P = sy.simplify(BMD_area_P * centroid_dist_P)

In [ ]:
t_A_C_P

In [ ]:
t_A_C_P.subs({L_AB_m: 4, L_BC_m: 4,})

#### 분포하중 $w0$에 의한 굽힘모멘트 면적<br>Bending moment area by distributed load $w0$

분포하중에 의한 굽힘모멘트 면적을 분해해서 구하는 대신, 브라켓 함수로 표시한다.<br>Instead of dividing the bending moment area, indicate the area using the bracket function.

브라켓 함수는 헤비사이드 함수로도 표시할 수 있다.<br>An engineer can describe a bracket function using the Heaviside function.

$$<x-a>^n = 1(x-a) \times (x-a)^n
$$

$x_A$~$x_B$에 분포하는 하중은 브라켓 함수 두개를 사용하여 표시할 수 있다.<br>It is possible to describe the load distributed between $x_A$ and $x_B$ using two bracket functions.

$$ 
\begin{align}
    \frac{d}{dx}SFD(x) &= w0<x-x_A> - w0<x-x_B>\\
                       &= w0[1(x-x_A) - 1(x-x_B)]
\end{align}
$$

In [ ]:
SFD_w0 = sy.integrate(w_AB_N_m * (sy.Heaviside(x_m - x_A_m) - sy.Heaviside(x_m - x_B_m)), x_m)

In [ ]:
sy.simplify(SFD_w0)

In [ ]:
splot.plot(SFD_w0.subs(s_d), (x_m, x_A_m, x_C_m.subs(s_d)), ylabel='SFD')

In [ ]:
BMD_w0 = sy.integrate(SFD_w0, x_m)

In [ ]:
BMD_w0.subs({L_AB_m: 4, L_BC_m: 4,})

`simplify`기능을 사용하면 브라켓 함수가 보다 분명히 드러난다.<br>`simplify` result reveal show bracket function more clearly.

In [ ]:
sy.simplify(BMD_w0.subs({L_AB_m: 4, L_BC_m: 4,}))

In [ ]:
splot.plot(BMD_w0.subs(s_d), (x_m, x_A_m, x_C_m.subs(s_d)), ylabel='BMD')

In [ ]:
BMDarea_w0 = sy.integrate(BMD_w0, (x_m, x_A_m, x_C_m))

In [ ]:
BMDarea_w0.subs({L_AB_m: 4, L_BC_m: 4,})

도심 거리<br>Centroid distance

In [ ]:
centroid_dist_w0 = sy.simplify(sy.integrate(BMD_w0 * x_m, (x_m, x_A_m, x_C_m))/ sy.integrate(BMD_w0, (x_m, x_A_m, x_C_m)))

In [ ]:
centroid_dist_w0.subs({L_AB_m: 4, L_BC_m: 4,})

선도 면적 $\times$ 도심 거리<br>Diagram area $\times$ centroid distance

In [ ]:
t_A_C_w0 = sy.simplify(BMDarea_w0 * centroid_dist_w0)

In [ ]:
t_A_C_w0.subs({L_AB_m: 4, L_BC_m: 4,})

#### 방정식<br>Equation

각각 집중하중과 분포하중에 의한 면적$\times$도심거리가 같다.<br>Respective area $\times$ (centroid distance) by concentrated load P and distributed load w0 are the same.

In [ ]:
eq = sy.Eq(t_A_C_P, t_A_C_w0)

In [ ]:
eq.subs({L_AB_m: 4, L_BC_m: 4,})

방정식의 해를 구한다.<br>Solve for P

In [ ]:
P_sol_list = sy.solve(eq, P_A_N)

In [ ]:
P_sol_list[0].subs({L_AB_m: 4, L_BC_m: 4,})

In [ ]:
P_sol_list[0].subs(s_d)

구해진 P(N) 는 다음과 같다.<br>P(N) would be as follows.

In [ ]:
float(P_sol_list[0].subs(s_d))